In [1]:
import os
import pandas as pd

version = "v01"
main_publisher = 'Merged'
script_dir = os.path.dirname(os.path.realpath('__file__'))
path = os.path.join(script_dir, '../Data/' + main_publisher + '/')
df_chapters = pd.read_csv(path + 'chapters.csv', sep = '|')
df_chapters.head()

,Cid,Eid,Sid,Text,Title,BeginTimestamp,EndTimestamp,Corpus
0,0,0,0,Professor Jonathan Holloway : \n “Fellow citi...,"Chapter 1. Frederick Douglass’ Speech, Deliver...",00:00:00,00:04:44,yaleocw
1,1,0,0,This class is about the post-emancipation Afri...,Chapter 2. What does it mean to be American?,00:04:44,00:07:56,yaleocw
2,2,0,0,"Near the Minuteman Park, there’s also a cemete...",Chapter 3. The Story of John Jack,00:07:56,00:15:43,yaleocw
3,3,0,0,"Two hundred years later, after John Jack’s att...",Chapter 4. The Linkage between Freedom and Cit...,00:15:43,00:19:25,yaleocw
4,4,0,0,We’re going to look for stories like this in a...,Chapter 5. The History of the Post-Emancipatio...,00:19:25,00:24:26,yaleocw


In [2]:
from concepts import text2RDF

df_chapters[237:].apply(lambda x : text2RDF(x['Text'], "../Output/Graphs/" + version + "/" + str(x['Cid'])), axis = 1)

Wikification for : 237
Graph generation for : 237
Saved in  ../Output/Graphs/v01/237.ttl
Wikification for : 238
Graph generation for : 238
Saved in  ../Output/Graphs/v01/238.ttl
Wikification for : 239
Graph generation for : 239
Saved in  ../Output/Graphs/v01/239.ttl
Wikification for : 240
Graph generation for : 240
Saved in  ../Output/Graphs/v01/240.ttl
Wikification for : 241
Graph generation for : 241
Saved in  ../Output/Graphs/v01/241.ttl
Wikification for : 242
Graph generation for : 242
Saved in  ../Output/Graphs/v01/242.ttl
Wikification for : 243
Graph generation for : 243
Saved in  ../Output/Graphs/v01/243.ttl
Wikification for : 244
Graph generation for : 244
Saved in  ../Output/Graphs/v01/244.ttl
Wikification for : 245
Graph generation for : 245
Saved in  ../Output/Graphs/v01/245.ttl
Wikification for : 246
Graph generation for : 246
Saved in  ../Output/Graphs/v01/246.ttl
Wikification for : 247
Graph generation for : 247
Saved in  ../Output/Graphs/v01/247.ttl
Wikification for : 24

237     ((http://en.wikipedia.org/wiki/So_(dairy_produ...
238     ((http://example.org/Chapter, http://purl.org/...
239     ((http://en.wikipedia.org/wiki/Torque, https:/...
240     ((http://en.wikipedia.org/wiki/Face, http://pu...
241     ((http://example.org/Chapter, http://purl.org/...
                              ...                        
4402    ((http://en.wikipedia.org/wiki/Crataegus_monog...
4403    ((http://en.wikipedia.org/wiki/First_Professio...
4404    ((http://en.wikipedia.org/wiki/Face, http://pu...
4405    ((http://en.wikipedia.org/wiki/Idea, http://ww...
4406    ((http://en.wikipedia.org/wiki/Crataegus_monog...
Length: 4170, dtype: object